## Example Usage

* Below are examples of syntax, using the tool to parse the example file provided

In [ ]:
## Parse header and print the metadata from the ASCII header
import importlib
import time
import parseTO as TO
importlib.reload(TO)
file = r'example_data\Met_Data100.dat'
pTO = TO.parseTO()
pTO.parse(file,mode=2)

example_data\Met_Data100.dat
Frames  4


['Read', 'example_data\\Met_Data100.dat']

In [177]:
## Read the file and output data to a 2D numpy array

import time
import parseTO as TO
file = r'example_data\Met_Data100.dat'
T1 = time.time()
TO = TO.parseTO()
TO.parse(file,mode=1)
print('Completed in: ',round((time.time()-T1),4),' s')

print(TO.Timestamp)
print(TO.Data)

example_data\Met_Data100.dat
Frames  4
Completed in:  0.0022  s
[1.7232498e+09 1.7232516e+09 1.7232534e+09 1.7232552e+09 1.7232570e+09
 1.7232588e+09 1.7232606e+09 1.7232624e+09 1.7232642e+09 1.7232660e+09
 1.7232678e+09 1.7232696e+09 1.7232714e+09 1.7232732e+09 1.7232750e+09
 1.7232768e+09 1.7232786e+09 1.7232804e+09 1.7232822e+09 1.7232840e+09
 1.7232858e+09 1.7232876e+09 1.7232894e+09 1.7232912e+09 1.7232930e+09
 1.7232948e+09 1.7232966e+09 1.7232984e+09 1.7233002e+09 1.7233020e+09
 1.7233038e+09 1.7233056e+09 1.7233074e+09 1.7233092e+09 1.7233110e+09
 1.7233128e+09 1.7233146e+09 1.7233164e+09 1.7233182e+09 1.7233200e+09
 1.7233218e+09 1.7233236e+09 1.7233254e+09 1.7233272e+09 1.7233290e+09
 1.7233308e+09 1.7233326e+09 1.7233344e+09 1.7233362e+09 1.7233380e+09
 1.7233398e+09 1.7233416e+09 1.7233434e+09 1.7233452e+09 1.7233470e+09
 1.7233488e+09 1.7233506e+09 1.7233524e+09 1.7233542e+09 1.7233560e+09]
[[ 0.77035666 13.89762688 13.94688225 ... 14.72953987 14.17568874
  14.1158638 ]
 [

In [178]:
## Read the file and output data to a pandas dataframe

import time
import parseTO as TO
file = r'example_data\Met_Data100.dat'
T1 = time.time()
TO = TO.parseTO()
TO.parse(file,mode=2)
print('Completed in: ',round((time.time()-T1),4),' s')

TO.Data.head()

example_data\Met_Data100.dat
Frames  4
Completed in:  0.0064  s


,Current_Avg,BattV_Avg,Voltage_Avg,NetRad_Avg,NetRad_Corrected_Avg,SlrW_Avg,AirTC_Avg,RH,BV_BP_Avg,BV_Temp_Avg,BV_Qual_Avg,Water_Level_m,Water_Temp_C,PTemp_C_Avg,TWGS_1_Avg,TWGS_2_Avg,TWGS_3_Avg
TIMESTAMP,,,,,,,,,,,,,,,,,
2024-08-10 00:30:00,0.770357,13.897627,13.946882,95.178658,97.047462,107.747437,13.525503,88.136353,1004.939697,13.033334,6.833444,0.050,8.2,15.832467,14.729540,14.175689,14.115864
2024-08-10 01:00:00,0.131140,13.716902,13.779596,35.430405,35.795799,79.168304,13.079014,93.205093,1005.091919,12.713916,6.835833,0.050,8.1,15.507368,14.635816,14.094248,13.979114
2024-08-10 01:30:00,0.190026,13.623872,13.685821,35.368614,35.594788,90.606110,12.589458,94.487923,1004.994995,12.037277,6.836416,0.050,8.1,14.914474,14.540340,14.015380,13.891110
2024-08-10 02:00:00,-0.230603,13.476729,13.547348,29.150885,29.233795,70.471649,12.395253,95.786430,1004.989380,11.500916,6.838000,0.050,8.1,14.445471,14.394469,13.881736,13.698770
2024-08-10 02:30:00,-0.722745,13.102289,13.191150,10.500364,10.551958,49.774025,12.546360,95.408714,1005.097717,11.319972,6.841944,0.052,8.1,14.255431,14.250505,13.760682,13.568392


In [ ]:
## Read the file, output data to a pandas dataframe, and save as a TOA5 formatted fiel in specified directory

import time
import parseTO as TO
file = r'example_data\Met_Data100.dat'
T1 = time.time()
TO = TO.parseTO()
TO.parse(file,mode=2,saveTo='C:/temp')
print('Completed in: ',round((time.time()-T1),4),' s')


example_data\Met_Data100.dat
Frames  4
Converted  example_data\Met_Data100_2024_08_10_0000.dat  to  C:/temp\Met_Data100_2024_08_10_0000.dat
Completed in:  0.0162  s


Speed for a given file is comparable to the cardconvert program.  But significant performance gains can be made for conversion by running the tool in parallel over a given directory, such as shown below:

In [ ]:
import os
import time
import parseTO as TO
from functools import partial
from multiprocessing import Pool

dpath = 'D:/GSC_Work/SCL_2024/20240724'
fileList = [os.path.join(dpath,f) for f in os.listdir(dpath) if '.dat' in f]
print('Processing: ',len(fileList),' files\n')

if __name__ == '__main__':
    with Pool(processes=4) as pool:
        T1 = time.time()
        TO = TO.parseTO(log=True)
        log = pool.map(partial(TO.parse,mode=2,saveTo='C:/temp'),fileList)
        print('Completed in: ',round((time.time()-T1),4),' s')
for result in log:
    print(result)

Processing:  250  files

Completed in:  333.0052  s
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data242_2024_07_20_2053.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data243_2024_07_20_2100.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data244_2024_07_20_2101.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data245_2024_07_20_2102.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data246_2024_07_20_2106.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data247_2024_07_20_2109.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data248_2024_07_20_2114.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data249_2024_07_20_2130.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data250_2024_07_20_2145.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data251_2024_07_20_2146.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data252_2024_07_20_2200.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240724\\Flux_Data253_2024_07_20_2208.dat']
['Read', 'D:/GSC_Work/SCL_2024/20240